In [1]:
import math
import numpy as np
import pandas as pd
import time

In [2]:
train_data = r'/home/hadoop/Downloads/counterfeit_train.csv'
test_data = r'/home/hadoop/Downloads/counterfeit_test.csv'

In [3]:
cct = pd.read_csv(train_data)
ccts = pd.read_csv(test_data)

In [4]:
a = list(cct.columns.values)

In [5]:
a

['Medicine_ID',
 'Counterfeit_Weight',
 'DistArea_ID',
 'Active_Since',
 'Medicine_MRP',
 'Medicine_Type',
 'SidEffect_Level',
 'Availability_rating',
 'Area_Type',
 'Area_City_Type',
 'Area_dist_level',
 'Counterfeit_Sales']

In [6]:
ccts['Counterfeit_Sales'] = pd.Series([0]* len(ccts['Medicine_ID']))

In [7]:
cct['dtype'] = pd.Series(['train']*len(cct['Medicine_ID']))

In [8]:
ccts['dtype'] = pd.Series(['test']*len(ccts['Medicine_ID']))

In [9]:
ccts = pd.DataFrame(data = ccts)

In [10]:
cct = pd.DataFrame(data = cct)

In [11]:
All_data = cct.append(ccts)

In [12]:
All_data.dtypes 

Medicine_ID             object
Counterfeit_Weight     float64
DistArea_ID             object
Active_Since             int64
Medicine_MRP           float64
Medicine_Type           object
SidEffect_Level         object
Availability_rating    float64
Area_Type               object
Area_City_Type          object
Area_dist_level         object
Counterfeit_Sales      float64
dtype                   object
dtype: object

In [13]:
All_data.isnull().sum()

Medicine_ID               0
Counterfeit_Weight     1463
DistArea_ID               0
Active_Since              0
Medicine_MRP              0
Medicine_Type             0
SidEffect_Level           0
Availability_rating       0
Area_Type                 0
Area_City_Type            0
Area_dist_level           0
Counterfeit_Sales         0
dtype                     0
dtype: int64

In [14]:
Mean_value = 0

In [15]:
All_data['Counterfeit_Weight'] = All_data['Counterfeit_Weight'].fillna(Mean_value)

In [16]:
All_data.isnull().sum()

Medicine_ID            0
Counterfeit_Weight     0
DistArea_ID            0
Active_Since           0
Medicine_MRP           0
Medicine_Type          0
SidEffect_Level        0
Availability_rating    0
Area_Type              0
Area_City_Type         0
Area_dist_level        0
Counterfeit_Sales      0
dtype                  0
dtype: int64

In [18]:
All_data['DistArea_ID'].value_counts()

Area027    935
Area013    932
Area046    930
Area049    930
Area035    930
Area045    929
Area018    928
Area017    926
Area010    555
Area019    528
Name: DistArea_ID, dtype: int64

In [20]:
for col in ['Medicine_Type','SidEffect_Level','Area_Type','Area_City_Type','Area_dist_level']:
    
    freq = All_data[col].value_counts()
    select_catgory = freq.index[freq> 1][:-1]
    
    print(col)
    
    for cat in select_catgory:
        name = col+'_'+str(cat)
        All_data[name] = (All_data[col]== cat).astype(int)
    del All_data[col] 

Medicine_Type
SidEffect_Level
Area_Type
Area_City_Type
Area_dist_level


In [22]:
All_data = All_data.drop(['DistArea_ID','Active_Since'],1)

In [23]:
cct = All_data.iloc[0:6818,]

In [24]:
cct.shape

(6818, 30)

In [25]:
ccts = All_data.iloc[6818:,]

In [26]:
cct =cct.drop(['dtype'],1)

In [27]:
ccts = ccts.drop(['dtype'],1)

In [28]:
ccts = ccts.drop(['Counterfeit_Sales'],1)

In [29]:
cct = cct.drop(['Medicine_ID'],1)

In [30]:
x = cct.drop(['Counterfeit_Sales'],1)

In [31]:
y = cct['Counterfeit_Sales']

In [32]:
from sklearn.cross_validation import train_test_split

/home/hadoop/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [33]:
training, testing = train_test_split(cct,test_size = 0.2,random_state = 2)

In [34]:
x_train = training.drop(['Counterfeit_Sales'],1)

In [35]:
y_train = training['Counterfeit_Sales']

In [36]:
x_test = testing.drop(['Counterfeit_Sales'],1)

In [37]:
y_test = testing['Counterfeit_Sales']

In [38]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge

In [39]:
lin = LinearRegression(Counterfeit_Sales~)

In [40]:
lin.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [72]:
model = sm.OLS(y_train, x_train).fit()

In [74]:
model_predict = model.predict(x_test)

In [79]:
final_model = sm.OLS(y, x).fit()

In [41]:
predicted = lin.predict(x_test)

In [75]:
residual = model_predict - y_test

In [76]:
MAE = abs(sum(residual))/len(y_test)

In [77]:
MAE

16.459412034148407

In [78]:
1-(MAE/1660)

0.9900846915456938

In [47]:
import statsmodels.api as sm

/home/hadoop/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [46]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [48]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(x_train.values, i) for i in range(x_train.shape[1])]
vif["features"] = x_train.columns

In [49]:
vif

,VIF Factor,features
0,10.109107,Counterfeit_Weight
1,6.889007,Medicine_MRP
2,3.617017,Availability_rating
3,13.515463,Medicine_Type_Antibiotics
4,13.706992,Medicine_Type_Hreplacements
5,10.336753,Medicine_Type_Antiseptics
6,9.960113,Medicine_Type_OralContraceptives
7,8.007571,Medicine_Type_Antipyretics
8,7.584838,Medicine_Type_Cardiac
9,7.805174,Medicine_Type_Mstablizers


In [50]:
ridge = Ridge(fit_intercept=True)

In [45]:
x_train.shape[1]

44

In [51]:
ridge.fit(x_train,y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [52]:
predicted_ridge = ridge.predict(x_test)

In [53]:
residual_ridge = predicted_ridge - y_test

In [54]:
MAE = abs(sum(residual_ridge))/len(y_test)

In [56]:
lasso = Lasso(fit_intercept=True)

In [57]:
lasso.fit(x_train,y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [58]:
prediction_lasso = lasso.predict(x_test)

In [59]:
residual_lasso = prediction_lasso-y_test

In [60]:
MAE = abs(sum(residual_lasso))/len(y_test)

In [61]:
final_ridge = ridge.fit(x,y)

In [62]:
ccts = ccts.drop(['Medicine_ID'],1)

In [80]:
final_prediction1 = final_model.predict(ccts) 

In [64]:
final_prediction = final_ridge.predict(ccts)

In [81]:
final_prediction1

0       2798.451448
1       4009.348190
2       1512.440615
3        133.311556
4        -90.819754
5       1038.352353
6       3973.366624
7       3525.027392
8       4905.772671
9       1686.640724
10      2986.029231
11      3959.890555
12      2543.643899
13      2649.003127
14      5160.737206
15      2774.834157
16      2800.423355
17      2693.952086
18      3115.141401
19      1051.186765
20      1930.907553
21      1909.533627
22      1127.796513
23      3277.161542
24      2499.199666
25      3083.708373
26      1853.125030
27      1065.297049
28      2984.867128
29      2478.025114
           ...     
1675     943.657488
1676    3003.719266
1677     705.575842
1678    1578.566022
1679    3205.588033
1680    1564.327539
1681    1690.192190
1682    2096.503562
1683    1832.105656
1684    2669.393164
1685    1850.278999
1686    3399.122693
1687    2376.665767
1688    2214.754041
1689     825.236212
1690    2899.294647
1691       2.160262
1692    3361.810570
1693    2528.075874


In [66]:
ccts1 = All_data.iloc[6818:,]

In [67]:
final_solution=pd.DataFrame(list(zip(ccts1['Medicine_ID'],list(final_prediction))),
                     columns=['Medicine_ID','Counterfeit_Sales'])

In [68]:
final_solution

,Medicine_ID,Counterfeit_Sales
0,HLZ81,2798.075163
1,ECE94,4011.052440
2,SAD14,1513.939714
3,EQV63,154.294971
4,AIR10,-113.503644
5,LIC37,1062.169651
6,EXZ14,3971.418713
7,UCB79,3525.864802
8,IDJ12,4908.792092
9,KLQ71,1675.774442


In [69]:
final_solution.to_csv('Ajay_yadav_P3.csv',index=False)
